<a href="https://colab.research.google.com/github/ImonHBnLT/Colab/blob/main/prog_linear_pesq_operacional_envio_produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gurobipy

In [9]:
# Estoque das fábricas
E = [[1200, 1100, 1000, 2000],
     [1500, 1400, 700, 1500]]

# Demanda dos clientes
D = [[600, 960, 550, 1100],
     [855, 380, 700, 970],
     [720, 1120, 250, 1360]]

# Custos de envio produto x fábrica x cliente
S = [[[0.30, 1.62, 3.20, 2.00], [0.55, 2.00, 3.00, 1.90]],
     [[0.50, 2.00, 4.00, 3.60], [0.21, 1.60, 3.00, 2.80]],
     [[0.42, 1.90, 3.85, 3.00], [0.32, 1.60, 3.00, 2.80]]]

# número de fábricas
n = len(E)
# número de clientes
m = len(D)
# número de produtos
o = len(E[0])

In [10]:
import gurobipy as gp

# Cria um modelo chamado distribuicao
distribuicao = gp.Model()

# Variáveis de decisão
x = distribuicao.addVars(n, m, o, vtype=gp.GRB.CONTINUOUS)

# Função objetivo:
distribuicao.setObjective(sum(S[j][i][k] * x[i,j,k] for i in range(n) for j in range(m) for k in range(o)), sense=gp.GRB.MINIMIZE
)

# Restrições
r1 = distribuicao.addConstrs(
    sum(x[i,j,k] for i in range(n)) == D[j][k]
    for j in range(m)
    for k in range(o)
)
r2 = distribuicao.addConstrs(
    sum(x[i,j,k] for j in range(m)) <= E[i][k]
    for i in range(n)
    for k in range(o)
)

# Suprimindo console output
distribuicao.setParam('OutputFlag', 0)

# Resolvendo o modelo
distribuicao.optimize()

# Verificando status do modelo
print("Status do modelo:  {}".format(distribuicao.status))
if distribuicao.status == 2:
  print("Valor ótimo = {:.2f}".format(distribuicao.objVal))

  for i in range(n):
    for j in range(m):
      for k in range(o):
          if x[i,j,k].x > 0:
            print("Da fábrica {}: para o cliente {} - enviar {:4.0f} unidades do produto {}."
              .format(i+1, j+1, x[i,j,k].x, k+1))
    print("---")
  print("Solução ótima encontrada!")
else:
  print("Solução ótima não encontrada!")

Status do modelo:  2
Valor ótimo = 18295.15
Da fábrica 1: para o cliente 1 - enviar  600 unidades do produto 1.
Da fábrica 1: para o cliente 1 - enviar  960 unidades do produto 2.
Da fábrica 1: para o cliente 1 - enviar  550 unidades do produto 3.
Da fábrica 1: para o cliente 1 - enviar 1100 unidades do produto 4.
Da fábrica 1: para o cliente 3 - enviar   75 unidades do produto 1.
Da fábrica 1: para o cliente 3 - enviar  100 unidades do produto 2.
Da fábrica 1: para o cliente 3 - enviar  250 unidades do produto 3.
Da fábrica 1: para o cliente 3 - enviar  830 unidades do produto 4.
---
Da fábrica 2: para o cliente 2 - enviar  855 unidades do produto 1.
Da fábrica 2: para o cliente 2 - enviar  380 unidades do produto 2.
Da fábrica 2: para o cliente 2 - enviar  700 unidades do produto 3.
Da fábrica 2: para o cliente 2 - enviar  970 unidades do produto 4.
Da fábrica 2: para o cliente 3 - enviar  645 unidades do produto 1.
Da fábrica 2: para o cliente 3 - enviar 1020 unidades do produto 2.
